In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
data = pd.read_parquet(r"C:\Users\alexd\Desktop\WorkSpace\NoteBook_Jupyter\Projet_DA_DST_Co2\Data_Clean\Co2_2024_Clean_V2.parquet")

data.columns = data.columns.str.strip()
data["Constructeur"] = data["Constructeur"].astype(str).str.strip().str.upper()
data

,ID,Country,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,IT,Erwltp (g/km),Fuel consumption,Electric range (km)
0,148534149,NL,FORD,FOCUS,1523.0,121.0,petrol,92.0,0.0,E13 32 37,2.10,5.3,0.0
1,148534150,NL,OPEL,VIVARO,2454.0,0.0,electric,100.0,244.0,None,0.00,0.0,343.0
2,148534151,NL,VOLVO,S60,2176.0,16.0,petrol/electric,186.0,163.0,None,0.00,0.7,93.0
3,148534152,NL,FORD,FOCUS,1549.0,123.0,petrol,92.0,0.0,E13 32 37,2.10,5.4,0.0
4,148534153,NL,TOYOTA,TOYOTA AYGO X,1088.0,108.0,petrol,53.0,0.0,None,0.00,4.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10779676,161049897,LU,ALPINA,BMW ALPINA D3S TOURING,2180.0,182.0,diesel,261.0,0.0,None,0.00,6.9,0.0
10779677,161051122,LU,VOLKSWAGEN,CALIFORNIA,2801.0,226.0,diesel,150.0,0.0,E1 29,1.02,8.6,0.0
10779678,161056425,LU,MERCEDES-BENZ,GLA 200 D,1761.0,141.0,diesel,110.0,0.0,E1 29,1.00,5.4,0.0
10779679,161061009,LU,JAGUAR,F-TYPE,1696.0,216.0,petrol,221.0,0.0,None,0.00,9.5,0.0


In [3]:
describe_q_poids = data[["WLTP_poids","Puissance_KW"]].describe()
describe_q_poids

,WLTP_poids,Puissance_KW
count,1.046366e+07,1.075620e+07
mean,1.668003e+03,1.166188e+02
std,3.795869e+02,6.434538e+01
min,3.170000e+02,8.000000e+00
25%,1.381000e+03,7.400000e+01
50%,1.590000e+03,1.000000e+02
75%,1.892000e+03,1.310000e+02
max,4.574000e+03,1.352000e+03


In [120]:
q25, q50, q75 = data["WLTP_poids"].quantile([0.25,0.50,0.75])
print(f"quantile de poids, 25% : {q25}")
print(f"quantile de poids, 50% : {q50}")
print(f"quantile de poids, 75% : {q75}")

quantile de poids, 25% : 1381.0
quantile de poids, 50% : 1590.0
quantile de poids, 75% : 1892.0


In [121]:
q25, q50, q75 = data["Puissance_KW"].quantile([0.25,0.50,0.75])
print(f"quantile de puissance, 25% : {q25}")
print(f"quantile de puissance, 50% : {q50}")
print(f"quantile de puissance, 75% : {q75}")

quantile de puissance, 25% : 74.0
quantile de puissance, 50% : 100.0
quantile de puissance, 75% : 131.0


In [4]:
regles = [
    ("OPE",   "OPEL"),
    ("SSANG", "SSANGYONG"),
    ("TOYO",  "TOYOTA"),
    ("MER",   "MERCEDES-BENZ"),
    ("VOLK",  "VOLKSWAGEN"),
    ("LAMBO", "LAMBORGHINI"),
    ("POR",   "PORSCHE"),
    ("AUD",   "AUDI"),
    ("BMW",   "BMW"),
    ("CITRO", "CITROËN"),
    ("FIAT",  "FIAT"),
    ("PEUG",  "PEUGEOT"),
    ("FORD",  "FORD"),
    ("RENAU", "RENAULT"),
    ("GREAT", "GREAT WALL"),
]

def clean_constructeur(x):
    if pd.isna(x):
      return x
    
    x = str(x)
    
    for prefix, new_name in regles:
        if x.startswith(prefix):
            return new_name
    return x

data["Constructeur"] = data["Constructeur"].apply(clean_constructeur)

In [5]:
data["Constructeur"] = data["Constructeur"].replace({"AUTOMOBILI LAMBORGHINI"        : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHINI S."     : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHINI S.P.A." : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHINI S.P.A"  : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHIN"         : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHINI SP"     : "LAMBORGHINI",
                                                     "AUTOMOBILI LAMBORGHINI SPA"    : "LAMBORGHINI",
                                                     "VOLSKWAGEN" : "VOLKSWAGEN",
                                                     "VW"         : "VOLKSWAGEN",
                                                     "CMS-AUTO/MERCEDES-BENZ" : "MERCEDES-BENZ",
                                                     "CMS AUTO/MERCEDES-BENZ" : "MERCEDES-BENZ",
                                                     "LANDROVER" : "LAND ROVER",
                                                     "NISSSAN" : "NISSAN",
                                                     "NISSA" : "NISSAN",
                                                     "ROLLS-ROYCE" : "ROLLS ROYCE",
                                                     "SSAYNGYONG" : "SSANGYONG",
                                                     "MC LAREN" : "MCLAREN",
                                                     "DFSK SERES SOKON"        : "DFSK MOTOR",
                                                     "DFSK SERES SOKON FENGON" : "DFSK MOTOR",
                                                     "DFSK SERES SOKON FENG"   : "DFSK MOTOR",
                                                     "DFSK"                    : "DFSK MOTOR",
                                                     "SERES"                   : "DFSK MOTOR",
                                                     "LYNK & CO"  : "LYNK&CO",
                                                     "LYNK AND CO" : "LYNK&CO",
                                                     "MITSUBISHI (J)" : "MITSUBISHI",
                                                     "CATERHAM CARS LTD." : "CATERHAM",
                                                     "ALLIED VEHICLES LTD." : "ALLIED VEHICLES LTD",
                                                     "CHERY JAECOO" : "CHERY",
                                                     "HYUNDAI GENESIS" : "HYUNDAI",
                                                     "INEOS GRENADIER" : "INEOS",
                                                     "KG MOBILITY" : "KGM",
                                                     "MG  ROEWE" : "MG",
                                                     "MG ROEWE"  : "MG",
                                                     "SOCIETE DES AUTOMOBILES ALPINE(SAA)" : "ALPINE"
                                                     })

In [6]:
constructeur_map = {
    "1103" : "SMART",
    "122"  : "VOLVO",
    "124"  : "VOLKSWAGEN",
    "15"   : "BMW",
    "166"  : "TOYOTA",
    "1683" : "SEAT",
    "17"   : "BENTLEY",
    "180"  : "PORSCHE",
    "189"  : "HONDA",
    "211"  : "AUDI",
    "162"  : "ALFA ROMEO",
    "170"  : "SUZUKI",
    "218"  : "LAMBORGHINI",
    "224"  : "MAZDA",
    "267"  : "JEEP",
    "27"   : "CITROËN",
    "350"  : "ROLLS ROYCE",
    "411"  : "NISSAN",
    "428"  : "FERRARI",
    "447"  : "SEAT",
    "45"   : "FIAT",
    "46"   : "FORD",
    "544"  : "MASERATI",
    "596"  : "DACIA",
    "66"   : "JAGUAR",
    "692"  : "HYUNDAI",
    "72"   : "LAND ROVER",
    "760"  : "KIA",
    "894"  : "SSANGYONG",
    "948"  : "LEXUS",
    "95"   : "PEUGEOT",
    "255"  : "LOTUS",
    "76"   : "MG",
    "79"   : "MORGAN",
    "93"   : "OPEL",
    "77"   : "MERCEDES-BENZ",
    "104"  : "RENAULT",
    "108"  : "SKODA",
    "1089" : "MINI",
    "77"   : "MERCEDES-BENZ",
    "104"  : "RENAULT",
    "108"  : "SKODA",
    "1089" : "MINI"
    }

data["Constructeur"] = data["Constructeur"].astype(str).replace(constructeur_map)

In [125]:
print(len(constructeur_map))

39


In [7]:
country_map = {
    "NL" : "Netherlands",
    "SE" : "Sweden",
    "NO" : "Norway",
    "FR" : "France",
    "DE" : "Germany",
    "ES" : "Spain",
    "AT" : "Austria",
    "GR" : "Greece",
    "IS" : "Iceland",
    "EE" : "Estonia",
    "SI" : "Slovenia",
    "HU" : "Hungary",
    "DK" : "Denmark",
    "RO" : "Romania",
    "IT" : "Italy",
    "IE" : "Ireland",
    "CZ" : "Czech Republic",
    "FI" : "Finland",
    "LT" : "Lithuania",
    "LV" : "Latvia",
    "HR" : "Croatia",
    "LU" : "Luxembourg",
    "PT" : "Portugal",
    "PL" : "Poland",
    "CY" : "Cyprus",
    "BG" : "Bulgaria",
    "MT" : "Malta",
    "BE" : "Belgium",
    "SK" : "Slovakia"
}


data["Pays"] = data["Country"].map(country_map)

In [8]:
data["Type_Carburant"] = data["Type_Carburant"].replace({"diesel/electric" : "Hybride Diesel",
                                                         "petrol/electric" : "Hybride Essence",
                                                         "diesel"          : "Diesel",
                                                         "petrol"          : "Essence",
                                                         "e85"             : "Essence",
                                                         "lpg"             : "Autre",
                                                         "hydrogen"        : "Autre",
                                                         "ng"              : "Autre",
                                                         "electric"        : "Electric"})

In [9]:
data['Etiquette_CO2'] = pd.cut(
    data['Co2_Emission(WLTP)'],
    bins=[-np.inf, 100, 120, 140, 160, 200, 250, np.inf],
    labels=['A', 'B', 'C', 'D', 'E', 'F', 'G']
)

In [10]:
q = data["WLTP_poids"].quantile([0.25, 0.5, 0.75])

q25 = q.loc[0.25]
q50 = q.loc[0.50]
q75 = q.loc[0.75]

def poids_quartile(x):
    if x <= q25:
        return "Q1_Léger"
    elif x <= q50:
        return "Q2_Moyen"
    elif x <= q75:
        return "Q3_Lourd"
    else:
        return "Q4_Très_lourd"

data["Poids_Quartile"] = data["WLTP_poids"].apply(poids_quartile)

In [11]:
q = data["Puissance_KW"].quantile([0.25, 0.5, 0.75])

q25 = q.loc[0.25]
q50 = q.loc[0.50]
q75 = q.loc[0.75]

def puissance_quartile(x):
    if x <= q25:
        return "Q1_Faible"
    elif x <= q50:
        return "Q2_Moyen"
    elif x <= q75:
        return "Q3_Puissant"
    else:
        return "Q4_Très_Puissant"

data["PuissanceKW_Quartile"] = data["Puissance_KW"].apply(puissance_quartile)

In [12]:
data = data[data["Type_Carburant"] != "unknown"]

In [13]:
data = data.drop(columns = ["Country","IT","Erwltp (g/km)"])

In [14]:
data = data.dropna(subset = ["Puissance_KW",
                             "WLTP_poids",
                             "Model",
                             "Co2_Emission(WLTP)",
                             "Etiquette_CO2"])

In [15]:
data = data.drop_duplicates()

In [20]:
def find_outliers(outliers):                             
    Q25 = outliers.quantile(0.25)                        
    Q75 = outliers.quantile(0.75)                        
    IQR = Q75 - Q25                                      
    lower_bound = Q25 - 1.5 * IQR                        
    upper_bound = Q75 + 1.5 * IQR                        
    return lower_bound, upper_bound                      


lower_bound, upper_bound = find_outliers(data["Puissance_KW"])      
print("Lower bound  =", lower_bound)                     
print("Upper bound  =", upper_bound)                     

display(data[data["Puissance_KW"] < lower_bound])                     
display(data[data["Puissance_KW"] > upper_bound])                     


Lower bound  = -10.0
Upper bound  = 214.0


,ID,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,Fuel consumption,Electric range (km),Pays,Etiquette_CO2,Poids_Quartile,PuissanceKW_Quartile


,ID,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,Fuel consumption,Electric range (km),Pays,Etiquette_CO2,Poids_Quartile,PuissanceKW_Quartile
14,148534163,TESLA,MODEL Y,2095.0,0.0,Electric,235.0,157.0,0.0,455.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
38,148534187,BMW,XM,2904.0,43.0,Hybride Essence,360.0,335.0,1.9,76.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
55,148534204,TESLA,MODEL Y,2087.0,0.0,Electric,255.0,155.0,0.0,565.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
67,148534216,TESLA,MODEL Y,2095.0,0.0,Electric,255.0,157.0,0.0,430.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
84,148534233,BMW,I4 EDRIVE40,2271.0,0.0,Electric,250.0,169.0,0.0,564.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10779637,161017682,MERCEDES-BENZ,GLS 450 D 4MATIC,2883.0,233.0,Diesel,270.0,0.0,8.9,0.0,Slovakia,F,Q4_Très_lourd,Q4_Très_Puissant
10779661,161020178,ASTON MARTIN,V12 VANTAGE,2009.0,315.0,Essence,515.0,0.0,13.9,0.0,Luxembourg,G,Q4_Très_lourd,Q4_Très_Puissant
10779671,161041199,FERRARI,812 COMPETIZIONE A,1911.0,370.0,Essence,607.0,0.0,16.3,0.0,Luxembourg,G,Q4_Très_lourd,Q4_Très_Puissant
10779676,161049897,ALPINA,BMW ALPINA D3S TOURING,2180.0,182.0,Diesel,261.0,0.0,6.9,0.0,Luxembourg,E,Q4_Très_lourd,Q4_Très_Puissant


In [21]:
def remove_outliers_iqr_multi(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

data = remove_outliers_iqr_multi(data, ["Puissance_KW", "WLTP_poids"])


In [22]:
data

,ID,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,Fuel consumption,Electric range (km),Pays,Etiquette_CO2,Poids_Quartile,PuissanceKW_Quartile
0,148534149,FORD,FOCUS,1523.0,121.0,Essence,92.0,0.0,5.3,0.0,Netherlands,C,Q2_Moyen,Q2_Moyen
1,148534150,OPEL,VIVARO,2454.0,0.0,Electric,100.0,244.0,0.0,343.0,Netherlands,A,Q4_Très_lourd,Q2_Moyen
2,148534151,VOLVO,S60,2176.0,16.0,Hybride Essence,186.0,163.0,0.7,93.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
3,148534152,FORD,FOCUS,1549.0,123.0,Essence,92.0,0.0,5.4,0.0,Netherlands,C,Q2_Moyen,Q2_Moyen
4,148534153,TOYOTA,TOYOTA AYGO X,1088.0,108.0,Essence,53.0,0.0,4.8,0.0,Netherlands,B,Q1_Léger,Q1_Faible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10779673,161045023,TOYOTA,TOYOTA C-HR,1574.0,111.0,Essence,72.0,0.0,4.9,0.0,Luxembourg,B,Q2_Moyen,Q1_Faible
10779674,161048306,VOLKSWAGEN,ID.4 PRO 210KW,2305.0,0.0,Electric,89.0,166.0,0.0,534.0,Luxembourg,A,Q4_Très_lourd,Q2_Moyen
10779675,161048485,MERCEDES-BENZ,CLE 220D,2102.0,130.0,Diesel,145.0,0.0,4.9,0.0,Luxembourg,C,Q4_Très_lourd,Q4_Très_Puissant
10779678,161056425,MERCEDES-BENZ,GLA 200 D,1761.0,141.0,Diesel,110.0,0.0,5.4,0.0,Luxembourg,D,Q3_Lourd,Q3_Puissant


In [23]:
data_clean = data.copy()

In [24]:

data_clean = data_clean[~(
    data_clean["Type_Carburant"].isin(["Essence","Diesel","Hybride Diesel","Hybride Essence"])
    & (data_clean["Fuel consumption"] == 0.0)
)]

data_clean = data_clean[~(
    data_clean["Type_Carburant"].isin(["Electric","Hybride Diesel","Hybride Essence"])
    & (data_clean["Electric range (km)"] == 0.0)
)]

data_clean = data_clean[~(
    data_clean["Type_Carburant"].isin(["Electric","Hybride Diesel","Hybride Essence"])
    & (data_clean["Conso_Wh/km"] == 0.0)
)]

In [25]:
data_clean

,ID,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,Fuel consumption,Electric range (km),Pays,Etiquette_CO2,Poids_Quartile,PuissanceKW_Quartile
0,148534149,FORD,FOCUS,1523.0,121.0,Essence,92.0,0.0,5.3,0.0,Netherlands,C,Q2_Moyen,Q2_Moyen
1,148534150,OPEL,VIVARO,2454.0,0.0,Electric,100.0,244.0,0.0,343.0,Netherlands,A,Q4_Très_lourd,Q2_Moyen
2,148534151,VOLVO,S60,2176.0,16.0,Hybride Essence,186.0,163.0,0.7,93.0,Netherlands,A,Q4_Très_lourd,Q4_Très_Puissant
3,148534152,FORD,FOCUS,1549.0,123.0,Essence,92.0,0.0,5.4,0.0,Netherlands,C,Q2_Moyen,Q2_Moyen
4,148534153,TOYOTA,TOYOTA AYGO X,1088.0,108.0,Essence,53.0,0.0,4.8,0.0,Netherlands,B,Q1_Léger,Q1_Faible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10779673,161045023,TOYOTA,TOYOTA C-HR,1574.0,111.0,Essence,72.0,0.0,4.9,0.0,Luxembourg,B,Q2_Moyen,Q1_Faible
10779674,161048306,VOLKSWAGEN,ID.4 PRO 210KW,2305.0,0.0,Electric,89.0,166.0,0.0,534.0,Luxembourg,A,Q4_Très_lourd,Q2_Moyen
10779675,161048485,MERCEDES-BENZ,CLE 220D,2102.0,130.0,Diesel,145.0,0.0,4.9,0.0,Luxembourg,C,Q4_Très_lourd,Q4_Très_Puissant
10779678,161056425,MERCEDES-BENZ,GLA 200 D,1761.0,141.0,Diesel,110.0,0.0,5.4,0.0,Luxembourg,D,Q3_Lourd,Q3_Puissant


In [141]:
mean_co2 = data_clean["Co2_Emission(WLTP)"].mean()
mean_co2

print(data_clean["ID"].nunique())

10394163


In [142]:
avg_co2_pays = (
    data_clean
    .groupby("Pays")["Co2_Emission(WLTP)"]
    .mean()
    .round(2)
    .reset_index(name = "Avg_Co2")
    .sort_values(by = "Avg_Co2", ascending=True)
)


avg_co2_pays

,Pays,Avg_Co2
20,Norway,11.85
27,Sweden,62.07
7,Finland,64.81
19,Netherlands,65.36
18,Malta,75.11
1,Belgium,79.99
22,Portugal,87.08
12,Iceland,92.51
8,France,94.12
17,Luxembourg,99.22


In [26]:
def nan(df):
    count=0
    for col in df.columns:
            if df[col].isna().sum() > 0:
                count=1
                print(f'"{col}": {df[col].isna().sum()} valeurs manquantes')
    if count == 0:
         print("pas de NaN.")
nan(data)

pas de NaN.


In [27]:
sample_V2 = data_clean.sample(n=900000, random_state= 42)


In [28]:
sample_V2 = sample_V2.drop_duplicates()

In [29]:
bins = [
    113,114,115,116,117,118,119,120,121,122,123,124,125,
    126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,
    141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,
    156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,
    171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,
    186,187,188,189,190,191,192,193,np.inf
]

labels = [
    50,75,100,125,150,170,190,210,230,240,260,280,310,
    330,360,400,450,540,650,740,818,898,983,1074,1172,1276,1386,1504,
    1629,1761,1901,2049,2205,2370,2544,2726,2918,3119,3331,3552,3784,
    4026,4279,4543,4818,5105,5404,5715,6126,6637,7248,7959,8770,9681,
    10692,11803,13014,14325,15736,17247,18858,20569,22380,24291,
    26302,28413,30624,32935,35346,37857,40468,43179,45990,48901,
    51912,55023,58134,61245,64356,67467,70000
]

sample_V2["Cout(€)"] = pd.cut(
    sample_V2["Co2_Emission(WLTP)"],
    bins=bins,
    labels=labels,
    right=False
).astype(float)

sample_V2["Cout(€)"] = sample_V2["Cout(€)"].fillna(0)

In [30]:
sample_V2 

,ID,Constructeur,Model,WLTP_poids,Co2_Emission(WLTP),Type_Carburant,Puissance_KW,Conso_Wh/km,Fuel consumption,Electric range (km),Pays,Etiquette_CO2,Poids_Quartile,PuissanceKW_Quartile,Cout(€)
4944536,152629072,SKODA,KAROQ,1466.0,137.0,Essence,85.0,0.0,6.0,0.0,Germany,C,Q2_Moyen,Q2_Moyen,1172.0
3833494,152763561,RENAULT,CLIO / 1.0 /,1259.0,117.0,Essence,67.0,0.0,5.2,0.0,Slovenia,B,Q1_Léger,Q1_Faible,150.0
10037326,159075324,PEUGEOT,3008,1526.0,144.0,Essence,96.0,0.0,6.4,0.0,Belgium,D,Q2_Moyen,Q2_Moyen,2049.0
9485139,159794023,VOLKSWAGEN,T-ROC,1591.0,128.0,Diesel,110.0,0.0,4.9,0.0,Italy,C,Q3_Lourd,Q3_Puissant,400.0
4786654,152721190,AUDI,A3 SPORTBACK,1654.0,138.0,Diesel,110.0,0.0,5.3,0.0,Germany,C,Q3_Lourd,Q3_Puissant,1276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2254214,145227961,TOYOTA,TOYOTA BZ4X,2082.0,0.0,Electric,150.0,147.0,0.0,503.0,France,A,Q4_Très_lourd,Q4_Très_Puissant,0.0
6603753,146384623,DACIA,JOGGER,1363.0,127.0,Essence,81.0,0.0,5.6,0.0,France,C,Q1_Léger,Q2_Moyen,360.0
6164017,146017365,DACIA,SANDERO,1279.0,128.0,Essence,67.0,0.0,5.6,0.0,France,C,Q1_Léger,Q1_Faible,400.0
9880809,160408919,OPEL,CORSA,1194.0,120.0,Essence,55.0,0.0,5.4,0.0,Italy,B,Q1_Léger,Q1_Faible,210.0
